In [16]:
#调入所需要的所以库
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer as si
from sklearn import tree
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [3]:
#用于加载数并进行一些简单的去问号处理
def read_data(file_path):
    data = pd.read_csv(file_path, header=None)
    x = pd.DataFrame(data)
    x = x.replace('?',np.nan)
    model = si(missing_values=np.nan ,strategy='most_frequent')
    x = model.fit_transform(x).astype(np.int)
    return x

In [5]:
train = read_data('train.csv')
test = read_data('test.csv')

#采用卡方检验进行特征的提取
model = SelectKBest(chi2, k=6)
model.fit(train[:,:-1], train[:,-1])
train_x = model.transform(train[:,:-1])
train_y = train[:,-1]
#划分测试集和训练集
x1 = train_x[:-1000]
y1 = train_y[:-1000]
x2 = train_x[-1000:]
y2 = train_y[-1000:]

In [17]:
#对特征进行升维
poly = PolynomialFeatures(degree=3, include_bias = False)
poly.fit(x1)
X_poly1 = poly.transform(x1)
X_poly2 = poly.transform(x2)

In [8]:
#决策树模型
clf = tree.DecisionTreeClassifier()
clf.fit(x1,y1)
clf.score(x2,y2)

0.29699999999999999

In [18]:
#搭建神经网络
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.line = nn.Sequential(
            nn.Linear(83,1024),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1024,512),            
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,9)
        )
    
    def forward(self, x):
        out = self.line(x)
        return out

In [19]:
train_x = torch.from_numpy(X_poly1).float()
train_y = torch.from_numpy(y1-1)

net = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())
out = net(train_x)
#训练模型
for i in range(500):
    out = net(train_x)
    loss = loss_fn(out, train_y.long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('%d/100 Loss: %.4f' % (i/100,loss.item()))

0/100 Loss: 14.4903
1/100 Loss: 1.8321
2/100 Loss: 1.7725
3/100 Loss: 1.7342
4/100 Loss: 1.7013


In [20]:
test_x = torch.from_numpy(X_poly2).float()
test_y = torch.from_numpy(y2-1)
pred = net(test_x)
_,pred = torch.max(pred,1)
q = (pred-test_y)
sum(q==0) #正确的个数

tensor(342)

In [ ]:
#最后的模型
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer as si
import numpy as np
import pandas as pd

#用于加载数并进行一些简单的去问号处理
def read_data(file_path):
    data = pd.read_csv(file_path, header=None)
    x = pd.DataFrame(data)
    x = x.replace('?',np.nan)
    model = si(missing_values=np.nan ,strategy='most_frequent')
    x = model.fit_transform(x).astype(np.int)
    return x


train = read_data('train.csv')
test = read_data('test.csv')
#32时效果最好
knn = KNeighborsClassifier(k=32)
knn.fit(train[:,:-1],train[:,-1])
knn.predict(test)